## Mapping Toronto Neighbourhood Data

In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
import numpy as np
import pandas as pd

In [3]:
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [4]:
import folium 

### Import Table Data and Converting It into a Nested List

In [5]:
res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [6]:
res.text

'\n<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"b57c96c7-f3db-46a7-b222-572b387cbf16","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":967921175,"wgRevisionId":967921175,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontar

In [7]:
soup = bs(res.text, "lxml")

In [8]:
table = soup.find('table',class_ ="wikitable sortable")

In [9]:
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

In [10]:
t_col = table.find_all('th')

In [11]:
t_rows = table.find_all('tr')

In [12]:
for th in t_col:
    print(th.text)

Postal Code

Borough

Neighborhood



In [13]:
for tr in t_rows:
    print(tr.text)


Postal Code

Borough

Neighborhood


M1A

Not assigned

Not assigned


M2A

Not assigned

Not assigned


M3A

North York

Parkwoods


M4A

North York

Victoria Village


M5A

Downtown Toronto

Regent Park, Harbourfront


M6A

North York

Lawrence Manor, Lawrence Heights


M7A

Downtown Toronto

Queen's Park, Ontario Provincial Government


M8A

Not assigned

Not assigned


M9A

Etobicoke

Islington Avenue, Humber Valley Village


M1B

Scarborough

Malvern, Rouge


M2B

Not assigned

Not assigned


M3B

North York

Don Mills


M4B

East York

Parkview Hill, Woodbine Gardens


M5B

Downtown Toronto

Garden District, Ryerson


M6B

North York

Glencairn


M7B

Not assigned

Not assigned


M8B

Not assigned

Not assigned


M9B

Etobicoke

West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale


M1C

Scarborough

Rouge Hill, Port Union, Highland Creek


M2C

Not assigned

Not assigned


M3C

North York

Don Mills


M4C

East York

Woodbine Heights


M5C

Downtown Toronto

S

In [14]:
# converting table data into nested list
rows = []
for tr in t_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    rows.append(row)

In [15]:
rows

[[],
 ['M1A\n', 'Not assigned\n', 'Not assigned\n'],
 ['M2A\n', 'Not assigned\n', 'Not assigned\n'],
 ['M3A\n', 'North York\n', 'Parkwoods\n'],
 ['M4A\n', 'North York\n', 'Victoria Village\n'],
 ['M5A\n', 'Downtown Toronto\n', 'Regent Park, Harbourfront\n'],
 ['M6A\n', 'North York\n', 'Lawrence Manor, Lawrence Heights\n'],
 ['M7A\n',
  'Downtown Toronto\n',
  "Queen's Park, Ontario Provincial Government\n"],
 ['M8A\n', 'Not assigned\n', 'Not assigned\n'],
 ['M9A\n', 'Etobicoke\n', 'Islington Avenue, Humber Valley Village\n'],
 ['M1B\n', 'Scarborough\n', 'Malvern, Rouge\n'],
 ['M2B\n', 'Not assigned\n', 'Not assigned\n'],
 ['M3B\n', 'North York\n', 'Don Mills\n'],
 ['M4B\n', 'East York\n', 'Parkview Hill, Woodbine Gardens\n'],
 ['M5B\n', 'Downtown Toronto\n', 'Garden District, Ryerson\n'],
 ['M6B\n', 'North York\n', 'Glencairn\n'],
 ['M7B\n', 'Not assigned\n', 'Not assigned\n'],
 ['M8B\n', 'Not assigned\n', 'Not assigned\n'],
 ['M9B\n',
  'Etobicoke\n',
  'West Deane Park, Princess Gard

In [16]:
rows_values = rows[1:]

In [17]:
rows_values

[['M1A\n', 'Not assigned\n', 'Not assigned\n'],
 ['M2A\n', 'Not assigned\n', 'Not assigned\n'],
 ['M3A\n', 'North York\n', 'Parkwoods\n'],
 ['M4A\n', 'North York\n', 'Victoria Village\n'],
 ['M5A\n', 'Downtown Toronto\n', 'Regent Park, Harbourfront\n'],
 ['M6A\n', 'North York\n', 'Lawrence Manor, Lawrence Heights\n'],
 ['M7A\n',
  'Downtown Toronto\n',
  "Queen's Park, Ontario Provincial Government\n"],
 ['M8A\n', 'Not assigned\n', 'Not assigned\n'],
 ['M9A\n', 'Etobicoke\n', 'Islington Avenue, Humber Valley Village\n'],
 ['M1B\n', 'Scarborough\n', 'Malvern, Rouge\n'],
 ['M2B\n', 'Not assigned\n', 'Not assigned\n'],
 ['M3B\n', 'North York\n', 'Don Mills\n'],
 ['M4B\n', 'East York\n', 'Parkview Hill, Woodbine Gardens\n'],
 ['M5B\n', 'Downtown Toronto\n', 'Garden District, Ryerson\n'],
 ['M6B\n', 'North York\n', 'Glencairn\n'],
 ['M7B\n', 'Not assigned\n', 'Not assigned\n'],
 ['M8B\n', 'Not assigned\n', 'Not assigned\n'],
 ['M9B\n',
  'Etobicoke\n',
  'West Deane Park, Princess Gardens, 

In [18]:
# removing unneccesary values from nested list
row_list = []
for i in range(len(rows_values)):
    r = []
    for j in range(len(rows_values[0])):
        r.append(rows_values[i][j].replace('\n' ,''))
    
    row_list.append(r)

In [19]:
row_list

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M

### Import Table Data in a Dataframe

In [20]:
# columns of the dataframes
df_col = ['Postal Code', 'Borough', 'Neighborhood']

In [21]:
df = pd.DataFrame(data = row_list, columns = df_col)

In [22]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [23]:
# filtering out all Not assigned values
df_Toronto = df[df['Borough'] != 'Not assigned']

In [24]:
# reassigining index values
df_Toronto.index = range(len(df_Toronto))

In [25]:
# data types in df2
df_Toronto.dtypes

Postal Code     object
Borough         object
Neighborhood    object
dtype: object

In [26]:
df_Toronto.head(15)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [27]:
df_Toronto.tail(15)

,Postal Code,Borough,Neighborhood
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores"
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
90,M1W,Scarborough,"Steeles West, L'Amoreaux West"
91,M4W,Downtown Toronto,Rosedale
92,M5W,Downtown Toronto,Stn A PO Boxes
93,M8W,Etobicoke,"Alderwood, Long Branch"
94,M9W,Etobicoke,"Northwest, West Humber - Clairville"
95,M1X,Scarborough,Upper Rouge
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown"
97,M5X,Downtown Toronto,"First Canadian Place, Underground city"


In [28]:
df_Toronto.shape

(103, 3)

In [29]:
# importing Given Geospatial_Coordinates data
geo_code = pd.read_csv('Geospatial_Coordinates.csv')

In [30]:
geo_code

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [31]:
geo_code.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [32]:
df_Toronto.dtypes

Postal Code     object
Borough         object
Neighborhood    object
dtype: object

In [33]:
# Final dataframe
Toronto_neighbour = pd.merge(df_Toronto, geo_code, on = 'Postal Code')

In [34]:
Toronto_neighbour.head(100)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944


In [35]:
Toronto_neighbour.shape

(103, 5)

In [36]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_neighbour['Borough'].unique()),
        Toronto_neighbour.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


### Plotting Geosppatial Data

In [38]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


In [39]:
# add markers to map
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Toronto_neighbour['Latitude'], Toronto_neighbour['Longitude'], 
                                           Toronto_neighbour['Borough'], Toronto_neighbour['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [40]:
DowntownT_data = Toronto_neighbour[Toronto_neighbour['Borough'] == 'Downtown Toronto'].reset_index(drop=True)

In [41]:
DowntownT_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


Geographical coordinates of Downtown Toronto.

In [42]:
address = 'Downtown Toronto, Ontario'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [43]:
map_DowntownT = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(DowntownT_data['Latitude'], DowntownT_data['Longitude'], DowntownT_data['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DowntownT)  
    
map_DowntownT

In [44]:
neighborhood_latitude = DowntownT_data.loc[5, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DowntownT_data.loc[5, 'Longitude'] # neighborhood longitude value

neighborhood_name = DowntownT_data.loc[5, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524, -79.3873826.


### Now, let's get the top 200 venues that are in Central Bay Street within a radius of 1000 meters

In [74]:
CLIENT_ID = '4UNV55N314Z4QZPSQWEYEACVJISYCCDEIVFKKF' # your Foursquare ID
CLIENT_SECRET = 'AZVSZ3T3JELVJBWDJU051NO5K5N1EY5BUCCOBD' # your Foursquare Secret
VERSION = '20200629' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4UNV55N314Z4QZPSQWEYEACVJISYCCDEIVFKKF
CLIENT_SECRET:AZVSZ3T3JELVJBWDJU051NO5K5N1EY5BUCCOBD


In [75]:
LIMIT = 200
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=4UNV55N314Z4QZPSQWEYEACVJISYCCDEIVFKKF&client_secret=AZVSZ3T3JELVJBWDJU051NO5K5N1EY5BUCCOBD&v=20200629&ll=43.6579524,-79.3873826&radius=1000&limit=200'

In [47]:
results = requests.get(url).json()

In [48]:
results

{'meta': {'code': 200, 'requestId': '5f110daa40a5a74076d2c4f8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 201,
  'suggestedBounds': {'ne': {'lat': 43.66695240900001,
    'lng': -79.37496583679133},
   'sw': {'lat': 43.64895239099999, 'lng': -79.39979936320866}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '537d4d6d498ec171ba22e7fe',
       'name': "Jimmy's Coffee",
       'location': {'address': '82 Gerrard Street W',
        'crossStreet': 'Gerrard & LaPlante',
        'lat': 43.65842123574496,
        'lng': -79.38561319551111,
 

In [49]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [50]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Hailed Coffee,Coffee Shop,43.658833,-79.383684
2,Neo Coffee Bar,Coffee Shop,43.660140,-79.385870
3,The Queen and Beaver Public House,Gastropub,43.657472,-79.383524
4,Somethin' 2 Talk About,Middle Eastern Restaurant,43.658395,-79.385338
...,...,...,...,...
95,Shangri-La Toronto,Hotel,43.649129,-79.386557
96,Omg! Oh My Gyro!,Souvlaki Shop,43.650064,-79.391104
97,Ryerson Image Centre,Art Gallery,43.657523,-79.379460
98,Apple Eaton Centre,Electronics Store,43.652968,-79.380670


In [51]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Downtown Toronto

In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [55]:
DowntownT_venues = getNearbyVenues(names=DowntownT_data['Neighborhood'],
                                   latitudes=DowntownT_data['Latitude'],
                                   longitudes=DowntownT_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [56]:
print(DowntownT_venues.shape)

(1240, 7)


In [57]:
DowntownT_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [58]:
DowntownT_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,70,70,70,70,70,70
Christie,17,17,17,17,17,17
Church and Wellesley,71,71,71,71,71,71
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [59]:
print('There are {} uniques categories.'.format(len(DowntownT_venues['Venue Category'].unique())))

There are 208 uniques categories.


## 3. Analyzing Each Neighborhood

In [60]:
# one hot encoding
DowntownT_onehot = pd.get_dummies(DowntownT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DowntownT_onehot['Neighborhood'] = DowntownT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DowntownT_onehot.columns[-1]] + list(DowntownT_onehot.columns[:-1])
DowntownT_onehot = DowntownT_onehot[fixed_columns]

DowntownT_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
DowntownT_onehot.shape

(1240, 208)

In [62]:
DowntownT_grouped = DowntownT_onehot.groupby('Neighborhood').mean().reset_index()
DowntownT_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.00,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.014286,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.00,0.00,0.014286,0.000000,0.014286,0.000000,0.014286,0.000000,0.000000
3,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.028169,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.014085,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,...,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,0.000000,...,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
7,"Garden District, Ryerson",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.01,0.00,0.00,0.000000,0.010000,0.000000,0.010000,0.010000,0.000000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.00,0.00,0.030769,0.000000,0.000000,0.046154,0.015385,0.000000,0.015385


In [63]:
DowntownT_grouped.shape

(19, 208)

In [64]:
num_top_venues = 5

for hood in DowntownT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DowntownT_grouped[DowntownT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2        Pharmacy  0.04
3        Beer Bar  0.04
4  Farmers Market  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3     Boat or Ferry  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.16
1   Italian Restaurant  0.06
2       Sandwich Place  0.06
3                 Café  0.04
4  Japanese Restaurant  0.04


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3    Candy Store  0.06
4      Nightclub  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.10
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.06
3           Restaurant  0.04
4              Gay Bar  0.0

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DowntownT_grouped['Neighborhood']

for ind in np.arange(DowntownT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DowntownT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Cheese Shop,Restaurant,Seafood Restaurant,Beer Bar,Pharmacy,Café
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Sculpture Garden,Boat or Ferry,Rental Car Location,Boutique,Airport
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Japanese Restaurant,Café,Salad Place,Department Store,Bubble Tea Shop,Bar,Burger Joint
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Diner,Baby Store,Italian Restaurant,Candy Store,Restaurant,Nightclub
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Pub,Men's Store,Mediterranean Restaurant,Hotel
5,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Deli / Bodega
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Gym,Japanese Restaurant,Restaurant,Deli / Bodega,Seafood Restaurant,Asian Restaurant,Steakhouse
7,"Garden District, Ryerson",Clothing Store,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Café,Japanese Restaurant,Hotel,Cosmetics Shop,Pizza Place,Electronics Store
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Sporting Goods Shop,Brewery,Scenic Lookout,Restaurant,Fried Chicken Joint,Music Venue
9,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Bar,Park,Dessert Shop,Bakery,Gaming Cafe,Pizza Place


# Clustering Neighbourhood Data

In [67]:
# set number of clusters
kclusters = 4

DowntownT_grouped_clustering = DowntownT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DowntownT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 3, 1, 1, 1, 1, 1, 1])

In [68]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

DowntownT_merged = DowntownT_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DowntownT_merged = DowntownT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

DowntownT_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Beer Store,Ice Cream Shop,Spa
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Gym,Creperie,Restaurant,Park,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Café,Japanese Restaurant,Hotel,Cosmetics Shop,Pizza Place,Electronics Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Restaurant,Café,Coffee Shop,Clothing Store,Cocktail Bar,American Restaurant,Cosmetics Shop,Japanese Restaurant,Breakfast Spot,Seafood Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Cheese Shop,Restaurant,Seafood Restaurant,Beer Bar,Pharmacy,Café


In [69]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DowntownT_merged['Latitude'], DowntownT_merged['Longitude'], DowntownT_merged['Neighborhood'], DowntownT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters

##### Cluster 1

In [70]:
DowntownT_merged.loc[DowntownT_merged['Cluster Labels'] == 0, DowntownT_merged.columns[[1] + list(range(5, DowntownT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,0,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


##### Cluster 2

In [71]:
DowntownT_merged.loc[DowntownT_merged['Cluster Labels'] == 1, DowntownT_merged.columns[[1] + list(range(5, DowntownT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Beer Store,Ice Cream Shop,Spa
1,Downtown Toronto,1,Coffee Shop,Diner,Gym,Creperie,Restaurant,Park,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
2,Downtown Toronto,1,Clothing Store,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Café,Japanese Restaurant,Hotel,Cosmetics Shop,Pizza Place,Electronics Store
3,Downtown Toronto,1,Restaurant,Café,Coffee Shop,Clothing Store,Cocktail Bar,American Restaurant,Cosmetics Shop,Japanese Restaurant,Breakfast Spot,Seafood Restaurant
4,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Cheese Shop,Restaurant,Seafood Restaurant,Beer Bar,Pharmacy,Café
5,Downtown Toronto,1,Coffee Shop,Sandwich Place,Italian Restaurant,Japanese Restaurant,Café,Salad Place,Department Store,Bubble Tea Shop,Bar,Burger Joint
7,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Gym,Thai Restaurant,Deli / Bodega,Bookstore,Cosmetics Shop
8,Downtown Toronto,1,Coffee Shop,Aquarium,Café,Hotel,Sporting Goods Shop,Brewery,Scenic Lookout,Restaurant,Fried Chicken Joint,Music Venue
9,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Salad Place,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Sushi Restaurant
10,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Deli / Bodega


##### Cluster 3

In [72]:
DowntownT_merged.loc[DowntownT_merged['Cluster Labels'] == 2, DowntownT_merged.columns[[1] + list(range(5, DowntownT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Sculpture Garden,Boat or Ferry,Rental Car Location,Boutique,Airport


##### Cluster 4

In [73]:
DowntownT_merged.loc[DowntownT_merged['Cluster Labels'] == 3, DowntownT_merged.columns[[1] + list(range(5, DowntownT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,3,Grocery Store,Café,Park,Athletics & Sports,Diner,Baby Store,Italian Restaurant,Candy Store,Restaurant,Nightclub
